In [2]:
from game_lists_site.models import db

db.rollback()

In [3]:
import pandas as pd
from peewee import JOIN

from game_lists_site.models import Game, User, UserGame, db
from game_lists_site.utils.utils import get_normalized_playtimes

normalized_playtimes = get_normalized_playtimes()

data = pd.DataFrame(
    [
        {
            "user_id": ug.user.id,
            "game_id": ug.game.id,
            "playtime": normalized_playtimes[ug.game.id][ug.user.id],
            "last_played": ug.last_played,
        }
        for ug in UserGame.select()
        .join(User)
        .order_by(User.id)
        .where(UserGame.playtime > 0)
        if ug.game.id in normalized_playtimes
        and ug.user.id in normalized_playtimes[ug.game.id]
    ]
)

In [4]:
data.head()

,user_id,game_id,playtime,last_played
0,76561197961114082,10,0.031343,2021-02-09 21:02:36
1,76561197961114082,550,0.013248,2017-01-31 00:04:20
2,76561197961114082,730,0.014099,2022-06-20 20:14:01
3,76561197961114082,381210,0.000158,2017-01-30 17:15:35
4,76561197961114082,744900,0.012754,2018-11-16 15:09:33


In [5]:
import numpy as np

time_80 = np.quantile(data.last_played.values, 0.8)
time_80

numpy.datetime64('2022-04-09T15:29:25.800000004')

In [6]:
train = data[data["last_played"] < time_80].copy()
val = data[data["last_played"] >= time_80].copy()

In [7]:
val.head()

,user_id,game_id,playtime,last_played
2,76561197961114082,730,0.014099,2022-06-20 20:14:01
20,76561198067514875,570,0.055063,2022-09-07 00:12:33
53,76561198083927294,311690,0.124988,2022-09-17 15:29:09
60,76561198083927294,305620,0.032716,2022-07-14 12:45:42
61,76561198083927294,250900,0.251476,2022-06-29 21:30:00


In [8]:
train_user_ids = np.sort(np.unique(train.user_id.values))
train_user_ids[:15]

array([76561197961114082, 76561198067514875, 76561198083927294,
       76561198083927315, 76561198083927332, 76561198083927365,
       76561198083927367, 76561198083927387, 76561198083927389,
       76561198083927421, 76561198083927452, 76561198083927499,
       76561198083927510, 76561198083927523, 76561198083927532])

In [9]:
num_users = len(train_user_ids)
num_users

1001

In [10]:
userid2idx = {o:i for i,o in enumerate(train_user_ids)}

In [11]:
train["user_id"] = train["user_id"].apply(lambda x: userid2idx[x])
train.head()

,user_id,game_id,playtime,last_played
0,0,10,0.031343,2021-02-09 21:02:36
1,0,550,0.013248,2017-01-31 00:04:20
3,0,381210,0.000158,2017-01-30 17:15:35
4,0,744900,0.012754,2018-11-16 15:09:33
5,1,4000,0.009341,2020-03-15 16:40:06


In [12]:
val["user_id"] = val["user_id"].apply(lambda x: userid2idx.get(x, -1)) # -1 for users not in training
val.head()

,user_id,game_id,playtime,last_played
2,0,730,0.014099,2022-06-20 20:14:01
20,1,570,0.055063,2022-09-07 00:12:33
53,2,311690,0.124988,2022-09-17 15:29:09
60,2,305620,0.032716,2022-07-14 12:45:42
61,2,250900,0.251476,2022-06-29 21:30:00


In [13]:
val = val[val["user_id"] >= 0].copy()
val.head()

,user_id,game_id,playtime,last_played
2,0,730,0.014099,2022-06-20 20:14:01
20,1,570,0.055063,2022-09-07 00:12:33
53,2,311690,0.124988,2022-09-17 15:29:09
60,2,305620,0.032716,2022-07-14 12:45:42
61,2,250900,0.251476,2022-06-29 21:30:00


In [14]:
# now encoding movieId
train_game_ids = np.sort(np.unique(train.game_id.values))
num_items = len(train_game_ids)
print(num_items)
train_game_ids[:15]

379


array([ 10,  70,  80, 220, 240, 300, 320, 400, 440, 500, 550, 570, 620,
       630, 730])

In [15]:
gameid2idx = {o:i for i,o in enumerate(train_game_ids)}
train["game_id"] = train["game_id"].apply(lambda x: gameid2idx[x])
val["game_id"] = val["game_id"].apply(lambda x: gameid2idx.get(x, -1))

In [16]:
val = val[val["game_id"] >= 0].copy()
val.head()

,user_id,game_id,playtime,last_played
2,0,14,0.014099,2022-06-20 20:14:01
20,1,11,0.055063,2022-09-07 00:12:33
53,2,189,0.124988,2022-09-17 15:29:09
60,2,182,0.032716,2022-07-14 12:45:42
61,2,124,0.251476,2022-06-29 21:30:00


In [17]:
val.shape

(3236, 4)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        # initlializing weights
        # self.user_emb.weight.data.uniform_(0,0.05)
        # self.item_emb.weight.data.uniform_(0,0.05)

    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u * v).sum(1)

    def print(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        print("u", u)
        print("v", u)

In [20]:
def valid_loss(model):
    model.eval()
    users = torch.LongTensor(val.user_id.values).cuda()
    items = torch.LongTensor(val.game_id.values).cuda()
    ratings = torch.FloatTensor(val.playtime.values).cuda()
    y_hat = model(users, items)
    loss = F.mse_loss(y_hat, ratings)
    return loss.item()

In [21]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for i in range(epochs):
        model.train()
        users = torch.LongTensor(train.user_id.values).cuda()
        items = torch.LongTensor(train.game_id.values).cuda()
        ratings = torch.FloatTensor(train.playtime.values).cuda()

        y_hat = model(users, items)
        loss = F.mse_loss(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        testloss = valid_loss(model)
        print("train loss %.3f valid loss %.3f" % (loss.item(), testloss))

In [22]:
model = MF(num_users, num_items, emb_size=1000).cuda()

In [23]:
train_epocs(model, epochs=1000, lr=1, wd=1e-5)

tensor(88.4299, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 1002.070 valid loss 2744.968
tensor(103.5466, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 58205.539 valid loss 2356.156
tensor(56.3728, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 20434.746 valid loss 2160.419
tensor(26.3887, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 11770.808 valid loss 2246.054
tensor(5.7647, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 5847.043 valid loss 3077.934
tensor(-13.9877, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 14976.073 valid loss 2975.926
tensor(-32.0360, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 6501.225 valid loss 2887.213
tensor(-31.8680, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.

In [24]:
train_epocs(model, epochs=1000, lr=0.1, wd=1e-5)

tensor(6.2266, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 26.812
tensor(4.7508, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 3.484 valid loss 28.992
tensor(5.0933, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 103.464 valid loss 24.920
tensor(5.3499, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 7.295 valid loss 23.370
tensor(5.0488, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 39.639 valid loss 21.844
tensor(4.5490, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 52.295 valid loss 20.006
tensor(4.0506, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 24.836 valid loss 18.552
tensor(3.7049, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 6.206 valid

In [25]:
train_epocs(model, epochs=1000, lr=0.01, wd=1e-5)

tensor(0.0597, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.043
tensor(0.0603, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.001 valid loss 0.044
tensor(0.0730, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.004 valid loss 0.044
tensor(0.0696, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.002 valid loss 0.043
tensor(0.0614, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.001 valid loss 0.043
tensor(0.0522, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.002 valid loss 0.043
tensor(0.0463, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.001 valid loss 0.043
tensor(0.0434, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.001 valid loss 0.043


In [26]:
train_epocs(model, epochs=1000, lr=0.001, wd=1e-5)

tensor(0.0066, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.036
tensor(0.0081, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.036
tensor(0.0074, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.036
tensor(0.0067, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.036
tensor(0.0066, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.036
tensor(0.0068, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.036
tensor(0.0070, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.036
tensor(0.0072, device='cuda:0', grad_fn=<SelectBackward0>) tensor(0.0551, device='cuda:0')
train loss 0.000 valid loss 0.036


In [50]:
from game_lists_site.models import Game, System, User, UserGame, user_data_dir


model = MF(num_users, num_items, emb_size=1000).cuda()
model.load_state_dict(torch.load(user_data_dir / "model.dat"))
model.eval()

MF(
  (user_emb): Embedding(1001, 1000)
  (item_emb): Embedding(379, 1000)
)

In [51]:
from game_lists_site.models import User

user = User.get_by_id(76561198091812571)
# user = User.get_by_id(76561198083927294)
# 76561198083927294
torch.save(model.state_dict(), "model.py")
users = torch.LongTensor([userid2idx[user.id]]).cuda()
games = list(gameid2idx.values())
items = torch.LongTensor(games).cuda()
result = model(users, items)
idx2gameid = {value: key for key, value in gameid2idx.items()}
result = {
    idx2gameid[game_idx.item()]: score.item() for score, game_idx in zip(result, items)
}

In [52]:
from game_lists_site.experiments.experiments1 import get_game_stats


games = {
    Game.get_by_id(key): value
    for key, value in sorted(result.items(), key=lambda item: item[1], reverse=True)
}
played_games = [
    ug.game
    for ug in UserGame.select()
    .where(UserGame.user == user)
    .where(UserGame.playtime > 0)
]
for game in games:
    if game not in played_games and get_game_stats(game).player_count > 10:
        print(game.name, games[game])

The Awesome Adventures of Captain Spirit 0.6580253839492798
Total War: WARHAMMER II 0.5392305254936218
KurtzPel 0.47865164279937744
RuneScape ® 0.25540199875831604
Shadowverse CCG 0.250907838344574
UNO 0.22962482273578644
Vampire Survivors 0.20753435790538788
Defiance 0.19989515841007233
Dying Light 2 Stay Human 0.19796031713485718
ELDEN RING 0.1773255467414856
Hand Simulator 0.16409832239151
Dead Island: Epidemic 0.15765494108200073
Black Mesa 0.13591395318508148
Red Dead Redemption 2 0.1323244273662567
SEGA Mega Drive and Genesis Classics 0.132247656583786
This War of Mine 0.12921354174613953
Arma 2: Operation Arrowhead 0.12362580746412277
Orwell: Keeping an Eye On You 0.12348823994398117
Line of Sight 0.12273726612329483
Poly Bridge 0.12169081717729568
Insurgency: Sandstorm 0.12014266848564148
Blender 0.11748165637254715
Stellaris 0.11720986664295197
Governor of Poker 3 0.11564776301383972
SpeedRunners 0.11113089323043823
Spiral Knights 0.1024954542517662
Creativerse 0.1023627519607